In [ ]:
import xml.etree.ElementTree as ET


In [ ]:
tree = ET.parse('cat1.gen.xml')
root = tree.getroot()

def tprint(obj,indent="->"):
    print(indent,obj.tag,obj.text.strip() if obj.text else "")
    for o in obj:
        tprint(o,"    "+indent)

tprint(root)
#print(ET.tostring(root[-1], encoding='unicode'))

In [ ]:
import xml.etree.ElementTree as ET
import re

class Parser:
    def __init__(self,path):
        self.root = ET.parse(path).getroot()
        self.keywords = {}
        self.phrases = []
        self.tab = ""
        for o in self.root:
            if o.tag == "key":
                self.keywords[o.get('value')] = []
                for oo in o.text.strip().split('\n'):
                    oo = re.sub("\s+"," ",oo.strip())
                    self.keywords[o.get('value')].append(oo)
            elif o.tag == "phrase":
                for oo in o.text.strip().split('\n'):
                    self.phrases.append(oo)
            elif o.tag == "tab":
                self.tab = ET.tostring(o[0], encoding='unicode')
                self.tab = re.sub("\s+" ,""   ,self.tab)
                self.tab = re.sub("\n"  ,""   ,self.tab)
                self.tab = re.sub(">"   ,"> " ,self.tab)
                self.tab = re.sub("<"   ," <" ,self.tab)
        print(self.keywords)
        print(self.phrases)
        print(self.tab)
    def get(self):
        return self.permute(self.noisify(self.phrases))
    def noisify(self,ilines):
        lines = []
        noise_level = 3
        for iline in ilines:
            for n1 in range(0,noise_level+1):
                for n2 in range(0,noise_level+1):
                    for n3 in range(0,noise_level+1):
                        for n4 in range(0,noise_level+1):
                            for n5 in range(0,noise_level+1):
                                n = 0
                                line = ""
                                for w in iline.split():
                                    if "_noise" in w:
                                        n+=1
                                        if w[-2].isnumeric():
                                            nmax=int(w[-2]) 
                                        else:
                                            nmax = 5
                                        if n1!=0 and n == 1:
                                            for _ in range(0,min(n1,nmax)):
                                                line += "_unk_ "
                                        if n2!=0 and n == 2:
                                            for _ in range(0,min(n2,nmax)):
                                                line += "_unk_ "
                                        if n3!=0 and n == 3:
                                            for _ in range(0,min(n3,nmax)):
                                                line += "_unk_ "
                                        if n4!=0 and n == 4:
                                            for _ in range(0,min(n4,nmax)):
                                                line += "_unk_ "
                                        if n5!=0 and n == 5:
                                            for _ in range(0,min(n5,nmax)):
                                                line += "_unk_ "
                                    else:
                                        line += w+" "
                                lines.append(line)
        return sorted(list(set(lines)))

    def permute(self,ilines):
        lines = []
        for iline in ilines:
            for key in self.keywords:
                for keyword in self.keywords[key]:
                    line = re.sub("_key_"," "+keyword+" ",iline)
                    tab = re.sub("_key_"," "+key+" ",self.tab)
                    ids = {}
                    v = 0

                    for i,w in enumerate(line.split()):
                        if w == "_var_":
                            v+=1
                            ids[v] = i+1
                    line = re.sub("_var_","_unk_",line)
                    tab2 = ""
                    for i,w in enumerate(tab.split()):
                        if "_var" in w:
                            tab2 += "_"+str(ids[int(w[-2])])+"_ "
                        else:
                            tab2 += w + " "
                    lines.append(line+"\t"+tab2)
        return lines

p = Parser("./generators/inc-dec.gen.xml")
with open("lol","w") as f:
    for l in p.get():
        f.write(l+"\n")

In [ ]:
'''
<key>
    <op value="opa">
        a1
        a2
    </op>
    <op value="opb">
        b1
        b2
    </op>
</key>
<key>
    <op>
        c1
        c2
    </op>
</key>
'''
keys = [
    {
        "add" : ["addition of","sum of"],
        "sub" : ["subtraction of","difference of"],
        "mul" : ["addition of","sum of"],
        "div" : ["division of","div of"],
    },
    {
        "" : ["set","let"]
    }
]
text = "_key2_ _var_ be _key1_ _var_ and _var_"
texttab = "_key1_ _var2_ _var3_ equ _var1_"
import re
def add(line,tab,keylist,keyidx):
    if keyidx >= len(keys):
        print(line,"\t\t",tab)
        return
    for op in keys[keyidx]:
        mykeylist = keylist+[op]
        mytab = re.sub("_key"+str(keyidx+1)+"_",op,tab)
        for k in keys[keyidx][op]:
            myline = re.sub("_key"+str(keyidx+1)+"_",k,line)
            for i,key in enumerate(keys):
                if i!=keyidx:
                    add(myline,mytab,mykeylist,keyidx+1)

add(text,texttab,[],0)